In [75]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# # Read sqlite query results into a pandas DataFrame
# con = sqlite3.connect("../Data/citibike.sqlite")
# df = pd.read_sql_query("SELECT * FROM allrides where date(starttime)==strftime("2019-07-01"), con)

# # Verify that result of SQL query is stored in the dataframe
# print(df.head())

# con.close()

In [104]:
sample = pd.read_csv('../Data/allrides_sample.csv')

- startInterval
- endInterval


### transform Dates

In [106]:
def reformat_date(df):
    datecol=['starttime','stoptime']
    for col in datecol:
        df[col]=pd.to_datetime(sample[col])
        hourcol=col+'_hour'
        mincol=col+'_min'
        datecol=col+'_date'
        df[datecol]=df[col].dt.date
        df[hourcol]=df[col].dt.hour
        df[mincol]=df[col].dt.minute
        df[mincol]=df[mincol].apply(lambda x: 0 if x < 30 else 30)
    df['season']=df['starttime'].dt.month.apply(lambda x: 'winter' if x <=2 else 'spring' if x<=5 else 
                                              'summer' if x<=8 else 'fall' if x<=11 else 'winter')

In [107]:
reformat_date(sample)